In [ ]:
import os
import yaml
import nbimporter
from datetime import datetime, date
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType,  DateType, TimestampType
from pyspark.sql.functions import to_date, to_timestamp
from lab_spark import create_spark_session
import yfinance as yf
from lab_pg_database_manager import PGDatabaseManager

In [ ]:
class raw_finance:
    
    def __init__(self, spark: SparkSession, equity_type, symbol_date_pairs):
        self.spark=spark
        self.equity_type=equity_type
        self.symbol_date_pairs=symbol_date_pairs

        
    def fetch_yfinance_record(self):
        try:
            symbol, start_date = self.symbol_date_pairs
            # Fetch stock data using yfinance
            quote = yf.Ticker(symbol)
            current_date = date.today()
            hist = quote.history(start=start_date, end=current_date)
    
            # Reset index to include Date as a column and format it
            hist.reset_index(inplace=True)
            hist["Date"] = hist["Date"].dt.date

            if self.equity_type = 'stock':
                # limit and stablize the fields of hist
                hist = hist[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']]
                
            # Add symbol and import_time to each row
            record_list = [
                tuple(row) + (symbol, import_time) for row in hist.itertuples(index=False)
            ]
    
            # print(record_list)
            return record_list
    
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return []  # Return an empty list on error

            
   def parallel_fetch_yfinance_record(self, record_schema):
        try:
            # Distribute (symbol, start_date) pairs across Spark workers
            param_rdd = spark.sparkContext.parallelize(self.symbol_date_pairs)    
            
            # Fetch data in parallel
            mapped_record_rdd = param_rdd.flatMap(fetch_yfinance_record)
    
            # Convert RDD to DataFrame
            result_df = self.spark.createDataFrame(mapped_record_rdd, schema=record_schema)
            # result_df = spark.createDataFrame(mapped_record_rdd)
    
            # Show or save the results
            # result_df.show()
            return result_df
        except Exception as e:
            print(f"Error paralleling fetch: {e}")z
            return self.spark.createDataFrame([]) 

## Load finalytics.stage.stock_eod_quote_yahoo and merge into fin.stock_eod_quote

In [ ]:
pg_table = "stage.stock_eod_quote_yahoo"  # Replace with the PostgreSQL table name

pg_truncate_script = f"truncate table {pg_table};"

pg_merge_script = "call fin.usp_load_stock_eod();"

load_pg_finalytics(iceberg_table, pg_url, pg_driver, pg_table,  pg_truncate_script, pg_merge_script)

In [ ]:
spark.sql(f'select count(*) from {iceberg_table}').show()

In [ ]:
spark.sql('select * from nessie.raw.stock_eod_yfinance').show(truncate=False)

In [ ]:
# print(symbol_start_date_pairs)